In [18]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import os
import pandas as pd
import numpy as np
from predict import preprocess, train_test_split, df_yielder

TRAIN_RATIO = 0.2
TIMESTEP = 12


In [20]:
def create_model(input_shape):
    
    model = Sequential()
    model.add(LSTM(units=50,return_sequences=True,input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))
    model.compile(optimizer='adam',loss='mean_squared_error')
    
    return model
   

In [8]:
df = pd.read_csv('data/yf_attributes/AAPL.csv')
scaled_df = preprocess(df)

In [17]:
def create_sequences(data, timestep):
    X, y = [], []
    for i in range(len(data) - timestep):
        X.append(data[i:i + timestep]) #Using all data between timesteps
        y.append(data[i + timestep, 4]) #Using next timesteps close data as target
    return np.array(X), np.array(y)

In [19]:
X, y = create_sequences(scaled_df, TIMESTEP)

split_point = int(TRAIN_RATIO * len(X))

X_train, X_test = X[:split_point], X[split_point:]
y_train, y_test = y[:split_point], y[split_point:]

print(X_train.shape)

In [ ]:
model = create_model()
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)